In [2]:
import pandas as pd
import numpy as np
import sys
import tqdm
import os
import glob

In [2]:
try:
    import google.colab
    colab = True
    from google.colab import drive
    ncbo_input = '/content/drive/MyDrive/NLP_files/CRAFT/results/bertopic_ncbo/*'
    embeddings_output_filepath = '/content/drive/MyDrive/NLP_files/CRAFT/data/embeddings/'
    drive.mount('/content/drive')
    device_name = 'cuda'
    sys.path.append('/content/drive/MyDrive/NLP/bert_embeddings')
except ImportError:
    colab = False
    ncbo_input = '../../0.RESULTS/*_ncbo/*'
    embeddings_output_filepath = '../../0.RESULTS/embeddings/'
    device_name = 'cpu'
from emb_helpers import * 


c:\users\krystian kurek\.conda\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
try:
    os.mkdir(embeddings_output_filepath)
except FileExistsError: 
    pass

In [4]:
from transformers import AutoTokenizer, AutoModel

biobert = AutoModel.from_pretrained("dmis-lab/biobert-v1.1").to(device_name)

In [5]:
files = glob.glob(ncbo_input)
df = pd.concat([pd.read_csv(file) for file in files])

In [17]:
df['ncbo_annotations_pairs'].iloc[0]#.apply(lambda x: sum([row[:2] for row in eval(x)], []))

"[['QUININE', 'heteroorganic entity', 'ANCESTOR', 'https://data.bioontology.org/ontologies/CHEBI/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_15854/ancestors'], ['TASTE', 'process', 'ANCESTOR', 'https://data.bioontology.org/ontologies/UBERON/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FGO_0050909/ancestors'], ['DOPAMINE', 'organic molecular entity', 'ANCESTOR', 'https://data.bioontology.org/ontologies/CL/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_18243/ancestors'], ['DOPAMINE', 'molecule', 'ANCESTOR', 'https://data.bioontology.org/ontologies/CHEBI/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_18243/ancestors'], ['DOPAMINE', 'heteroorganic entity', 'ANCESTOR', 'https://data.bioontology.org/ontologies/CL/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_18243/ancestors'], ['DOPAMINE', 'p-block molecular entity', 'ANCESTOR', 'https://data.bioontology.org/ontologies/CHEBI/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_59905/ancestors'], ['DOP

In [18]:
df['ncbo_annotations_pairs'].apply(lambda x: sum([row[:2] for row in eval(x)], [])).iloc[0]

['QUININE',
 'heteroorganic entity',
 'TASTE',
 'process',
 'DOPAMINE',
 'organic molecular entity',
 'DOPAMINE',
 'molecule',
 'DOPAMINE',
 'heteroorganic entity',
 'DOPAMINE',
 'p-block molecular entity',
 'DOPAMINE',
 'material entity',
 'QUININE',
 'organic molecular entity',
 'QUININE',
 'quinoline alkaloid',
 'LEARNING',
 'response to stimulus',
 'DOPAMINE',
 'organic hydroxy compound',
 'DOPAMINE',
 'polyatomic cation',
 'MEMORY',
 'cognition',
 'DOPAMINE',
 'oxygen molecular entity',
 'DOPAMINE',
 'polyatomic entity',
 'TASTE',
 'nervous system process',
 'MEMORY',
 'learning or memory',
 'DOPAMINE',
 'benzenediols',
 'DOPAMINE',
 'hydroxides',
 'DOPAMINE',
 'ion',
 'MEMORY',
 'behavior',
 'LEARNING',
 'occurrent',
 'MEMORY',
 'multicellular organismal process',
 'MEMORY',
 'system process',
 'DOPAMINE',
 'catechols',
 'DOPAMINE',
 'hydrogen molecular entity',
 'DOPAMINE',
 'independent continuant',
 'DOPAMINE',
 'ammonium ion derivative',
 'DOPAMINE',
 'aralkylamino compound',

In [19]:
df['ncbo_annotations_pairs'] = df['ncbo_annotations_pairs'].apply(lambda x: sum([row[:2] for row in eval(x)], []))
all_words_to_embed = list(set(sum(df['ncbo_annotations_pairs'], [])))
all_words_to_embed = pd.DataFrame(all_words_to_embed, columns=['words'])
embeddings = return_embeddings_for_concepts(all_words_to_embed['words'],biobert, 'cpu')
embeddings['words'] = all_words_to_embed['words']
embeddings[['words'] + list(range(len(embeddings.columns)-1))]
basename = 'ncbo_embeddings.csv'
print(basename)
embeddings.to_csv(os.path.join(embeddings_output_filepath, basename), index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 495/495 [00:18<00:00, 26.86it/s]


ncbo_embeddings.csv
